In [ ]:
import sys
import localSettings as ls
print(ls.main_path)

In [ ]:
main_path = ls.main_path
sys.path.append(main_path)

In [ ]:
from datetime import datetime
now = datetime.now()
date_time = now.strftime("%m%d%Y")
print("date and time:",date_time)

In [ ]:
SHRCALIBFACTOR = 1.00

In [ ]:
# USING BDT?
USEBDT = False

In [ ]:
# Doing Run3?
ISRUN3 = False

In [ ]:
# scale to MCC8 CV?
MCC8WEIGHTS = False

In [ ]:
import plotter
import importlib
importlib.reload(plotter)
import uproot
import matplotlib.pylab as pylab
import numpy as np
import math
from sklearn.model_selection import train_test_split
import pickle
import xgboost as xgb
import nue_booster 
importlib.reload(nue_booster)
import awkward

params = {
    'axes.labelsize': 'x-large',
    'axes.titlesize': 'x-large',
    'xtick.labelsize': 'x-large',
    'ytick.labelsize': 'x-large'
}
pylab.rcParams.update(params)

In [ ]:
fold = "nuselection"
tree = "NeutrinoSelectionFilter"

BNB = ""
EXT = ""
NU  = ""
NUE = ""
DRT = ""
NCPI0 = ""
CCPI0 = ""
CCNOPI = ""
NCCPI = ""
NCNOPI = ""

if ISRUN3:
    BNB = 'data_bnb_mcc9.1_v08_00_00_25_reco2_G1_beam_good_reco2_1e19'
    EXT = 'data_extbnb_mcc9.1_v08_00_00_25_reco2_G1_all_reco2'
    NU  = 'prodgenie_bnb_nu_uboone_overlay_mcc9.1_v08_00_00_26_filter_run3_reco2_G_reco2'
    NUE = 'prodgenie_bnb_intrinsice_nue_uboone_overlay_mcc9.1_v08_00_00_26_run3_reco2_reco2'
    DRT = 'prodgenie_bnb_dirt_overlay_mcc9.1_v08_00_00_26_run3_reco2_reco2'
    NCPI0 = 'prodgenie_nc_pi0_uboone_overlay_mcc9.1_v08_00_00_26_run3_G_reco2'
    CCPI0 = 'prodgenie_cc_pi0_uboone_overlay_v08_00_00_26_run3_G_reco2'
    CCNOPI = 'prodgenie_CCmuNoPi_overlay_mcc9_v08_00_00_33_run3_reco2_reco2'
    NCCPI = 'prodgenie_NCcPiNoPi0_overlay_mcc9_v08_00_00_33_New_run3_reco2_reco2'
    NCNOPI = 'prodgenie_ncnopi_overlay_mcc9_v08_00_00_33_new_run3_reco2_reco2'
    
else:
    BNB = 'data_bnb_mcc9.1_v08_00_00_25_reco2_C1_beam_good_reco2_5e19'
    EXT = 'data_extbnb_mcc9.1_v08_00_00_25_reco2_C_all_reco2'
    NU  = 'prodgenie_bnb_nu_uboone_overlay_mcc9.1_v08_00_00_26_filter_run1_reco2_reco2'
    NUE = 'prodgenie_bnb_intrinsice_nue_uboone_overlay_mcc9.1_v08_00_00_26_run1_reco2_reco2'
    DRT = 'prodgenie_bnb_dirt_overlay_mcc9.1_v08_00_00_26_run1_reco2_reco2'
    NCPI0 = 'prodgenie_nc_pi0_uboone_overlay-v08_00_00_26_run1_reco2_reco2'
    CCPI0 = 'prodgenie_cc_pi0_uboone_overlay_v08_00_00_26_run1_reco2'
    CCNOPI = 'prodgenie_CCmuNoPi_overlay_mcc9_v08_00_00_33_run1_reco2_reco2'
    NCCPI = 'prodgenie_NCcPiNoPi0_overlay_mcc9_v08_00_00_33_run1_reco2_reco2'
    NCNOPI = 'prodgenie_ncnopi_overlay_mcc9_v08_00_00_33_run1_reco2_reco2'
    
mc = uproot.open(ls.ntuple_path+NU+".root")[fold][tree]
ncpi0 = uproot.open(ls.ntuple_path+NCPI0+".root")[fold][tree]
ccpi0 = uproot.open(ls.ntuple_path+CCPI0+".root")[fold][tree]
ccnopi = uproot.open(ls.ntuple_path+CCNOPI+".root")[fold][tree]
nccpi = uproot.open(ls.ntuple_path+NCCPI+".root")[fold][tree]
ncnopi = uproot.open(ls.ntuple_path+NCNOPI+".root")[fold][tree]
nue = uproot.open(ls.ntuple_path+NUE+".root")[fold][tree]
data = uproot.open(ls.ntuple_path+BNB+".root")[fold][tree]
ext = uproot.open(ls.ntuple_path+EXT+".root")[fold][tree]
dirt = uproot.open(ls.ntuple_path+DRT+".root")[fold][tree]
lee = uproot.open(ls.ntuple_path+NUE+".root")[fold][tree]

uproot_v = [lee,mc,ncpi0,ccpi0,ccnopi,nccpi,ncnopi,nue,ext,data,dirt]

variables = [
    "shr_dedx_Y", "shr_bkt_pdg", "p", "pt", "selected", "nu_pdg", "shr_theta",
    "slpdg", "trk_score_v", "backtracked_pdg", # modified from shr_score_v
    "shr_pfp_id_v", "category", "shr_tkfit_dedx_Y",
    "shr_tkfit_gap10_dedx_U","shr_tkfit_gap10_dedx_V","shr_tkfit_gap10_dedx_Y",
    "shr_tkfit_2cm_dedx_U","shr_tkfit_2cm_dedx_V","shr_tkfit_2cm_dedx_Y",
    #"shr_energy_tot", 
    "trk_energy_tot", "shr_hits_tot", "ccnc", "trk_chipr",
    "trk_bkt_pdg", "hits_ratio", "n_tracks_contained", 
    "crtveto","crthitpe","_closestNuCosmicDist",
    "NeutrinoEnergy2",
    #"run","sub","evt",
    "CosmicIP",
    "nu_flashmatch_score","best_cosmic_flashmatch_score","best_obviouscosmic_flashmatch_score",
    #"trk_pfp_id",
    "shrmoliereavg","shrmoliererms",
    "shr_tkfit_npointsvalid","shr_tkfit_npoints", # fitted vs. all hits for shower
    "shrclusfrac0","shrclusfrac1","shrclusfrac2", # track-fitted hits / all hits
    "trkshrhitdist2", # "trkshrhitdist0","trkshrhitdist1", distance between track and shower in 2D
    "shrsubclusters0","shrsubclusters1","shrsubclusters2", # number of sub-clusters in shower
    "trk_llr_pid_score_v", # trk-PID score
    #"pi0_energy2_Y", # pi0 tagger variables
    "reco_nu_vtx_sce_x","reco_nu_vtx_sce_y","reco_nu_vtx_sce_z",
    "nproton", "nu_e", "n_showers_contained", "shr_distance", "trk_distance",
    "hits_y", "shr_pz", "shr_energy", "shr_dedx_U", "shr_dedx_V", "shr_phi", "trk_phi", "trk_theta",
    "shr_tkfit_dedx_U", "shr_tkfit_dedx_V", "run", "sub", "evt", "nproton", "trk_pid_chipr_v",
    "trk_len", "mc_pdg", "slnunhits", "slnhits", "shr_score", "trk_score", "trk_hits_tot",
    "true_e_visible", "matched_E", "shr_bkt_E", "trk_bkt_E", "trk_energy", "tksh_distance", "tksh_angle",
    "npi0","npion","pion_e","muon_e","pi0truth_elec_etot",
    "pi0_e", "shr_energy_tot_cali", "shr_dedx_Y_cali", "evnunhits", "nslice", "interaction",
    "slclustfrac", "reco_nu_vtx_x", "reco_nu_vtx_y", "reco_nu_vtx_z","contained_fraction",
    "secondshower_Y_nhit","secondshower_Y_vtxdist","secondshower_Y_dot","secondshower_Y_dir","shrclusdir2"
]
#make the list unique
variables = list(set(variables))
print(variables)

if not ISRUN3:
    variables.remove("_closestNuCosmicDist")
    variables.remove("crtveto")
    variables.remove("crthitpe")
    
WEIGHTS = ["weightSpline","weightTune","weightSplineTimesTune"]#, "weightsGenie", "weightsFlux", "weightsReint"]
WEIGHTSLEE = ["weightSpline","weightTune","weightSplineTimesTune", "leeweight"]#, "weightsGenie", "weightsFlux", "weightsReint"]

nue = nue.pandas.df(variables + WEIGHTS, flatten=False)
mc = mc.pandas.df(variables + WEIGHTS, flatten=False)
ncpi0 = ncpi0.pandas.df(variables + WEIGHTS, flatten=False)
ccpi0 = ccpi0.pandas.df(variables + WEIGHTS, flatten=False)
ccnopi = ccnopi.pandas.df(variables + WEIGHTS, flatten=False)
nccpi = nccpi.pandas.df(variables + WEIGHTS, flatten=False)
ncnopi = ncnopi.pandas.df(variables + WEIGHTS, flatten=False)
data = data.pandas.df(variables, flatten=False)
ext = ext.pandas.df(variables, flatten=False)
dirt = dirt.pandas.df(variables + WEIGHTS, flatten=False)
lee = lee.pandas.df(variables + WEIGHTSLEE, flatten=False)

In [ ]:
df_v = [lee,mc,ncpi0,ccpi0,ccnopi,nccpi,ncnopi,nue,dirt]

for i,df in enumerate(df_v):
    
    df.loc[ df['weightTune'] <= 0, 'weightTune' ] = 1.
    df.loc[ df['weightTune'] == np.inf, 'weightTune' ] = 1.
    df.loc[ df['weightTune'] > 100, 'weightTune' ] = 1.
    df.loc[ np.isnan(df['weightTune']) == True, 'weightTune' ] = 1.
    df.loc[ df['weightSplineTimesTune'] <= 0, 'weightSplineTimesTune' ] = 1.
    df.loc[ df['weightSplineTimesTune'] == np.inf, 'weightSplineTimesTune' ] = 1.
    df.loc[ df['weightSplineTimesTune'] > 100, 'weightSplineTimesTune' ] = 1.
    df.loc[ np.isnan(df['weightSplineTimesTune']) == True, 'weightSplineTimesTune' ] = 1.
    #df['weightSpline']  = df['weightSpline']  * df['weightTune']

In [ ]:
# adjust from MCC9 CV to MCC8 CV

if (MCC8WEIGHTS == True):

    # scaling for QE
    CV_bins = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,5.0]
    CV_scaling = [2.5,2.0,1.7,1.45,1.3,1.25,1.175,1.15,1.14,1.1]
    # scaling for RES

    mc.loc[ (mc['interaction'] == 10), 'weightSpline' ] = 2 * mc['weightSpline']
    ncpi0.loc[ (ncpi0['interaction'] == 10), 'weightSpline' ] = 2 * ncpi0['weightSpline']
    ccpi0.loc[ (ccpi0['interaction'] == 10), 'weightSpline' ] = 2 * ccpi0['weightSpline']
    ccnopi.loc[ (ccnopi['interaction'] == 10), 'weightSpline' ] = 2 * ccnopi['weightSpline']
    nccpi.loc[ (nccpi['interaction'] == 10), 'weightSpline' ] = 2 * nccpi['weightSpline']
    ncnopi.loc[ (ncnopi['interaction'] == 10), 'weightSpline' ] = 2 * ncnopi['weightSpline']
    nue.loc[ (nue['interaction'] == 10), 'weightSpline' ] = 2 * nue['weightSpline']
    lee.loc[ (lee['interaction'] == 10), 'weightSpline' ] = 2 * lee['weightSpline']
    dirt.loc[ (dirt['interaction'] == 10), 'weightSpline' ] = 2 * dirt['weightSpline']

    for i, CV_bin in enumerate(CV_bins):

        if i == 0:
            continue

        mc.loc[ (mc['nu_e'] > CV_bins[i-1]) & (mc['nu_e'] < CV_bins[i]) & (mc['interaction'] == 0), 'weightSpline' ] = CV_scaling[i-1] * mc['weightSpline']
        nue.loc[ (nue['nu_e'] > CV_bins[i-1]) & (nue['nu_e'] < CV_bins[i]) & (nue['interaction'] == 0), 'weightSpline' ] = CV_scaling[i-1] * nue['weightSpline']
        ncpi0.loc[ (nc['nu_e'] > CV_bins[i-1]) & (ncpi0['nu_e'] < CV_bins[i]) & (ncpi0['interaction'] == 0), 'weightSpline' ] = CV_scaling[i-1] * ncpi0['weightSpline']
        ccpi0.loc[ (ccpi0['nu_e'] > CV_bins[i-1]) & (ccpi0['nu_e'] < CV_bins[i]) & (ccpi0['interaction'] == 0), 'weightSpline' ] = CV_scaling[i-1] * ccpi0['weightSpline']
        ccnopi.loc[ (ccnopi['nu_e'] > CV_bins[i-1]) & (ccnopi['nu_e'] < CV_bins[i]) & (ccnopi['interaction'] == 0), 'weightSpline' ] = CV_scaling[i-1] * ccnopi['weightSpline']
        nccpi.loc[ (nccpi['nu_e'] > CV_bins[i-1]) & (nccpi['nu_e'] < CV_bins[i]) & (nccpi['interaction'] == 0), 'weightSpline' ] = CV_scaling[i-1] * nccpi['weightSpline']
        ncnopi.loc[ (ncnopi['nu_e'] > CV_bins[i-1]) & (ncnopi['nu_e'] < CV_bins[i]) & (ncnopi['interaction'] == 0), 'weightSpline' ] = CV_scaling[i-1] * ncnopi['weightSpline']
        lee.loc[ (lee['nu_e'] > CV_bins[i-1]) & (lee['nu_e'] < CV_bins[i]) & (lee['interaction'] == 0), 'weightSpline' ] = CV_scaling[i-1] * lee['weightSpline']
        dirt.loc[ (dirt['nu_e'] > CV_bins[i-1]) & (dirt['nu_e'] < CV_bins[i]) & (dirt['interaction'] == 0), 'weightSpline' ] = CV_scaling[i-1] * dirt['weightSpline']



In [ ]:
# how to get the LLR-PID value for the "track candidate" (proton for nue selection, muon for numu)
# can be done for any variable
# code from Giuseppe!

df_v = [lee,mc,ncpi0,ccpi0,ccnopi,nccpi,ncnopi,nue,ext,data,dirt]
#df_v = [lee,mc,nue,ext,data,dirt]

for i,df in enumerate(df_v):
    up = uproot_v[i]
    trk_llr_pid_v = up.array('trk_llr_pid_score_v')
    trk_id = up.array('trk_id')-1 # I think we need this -1 to get the right result
    trk_llr_pid_v_sel = awkward.fromiter([pidv[tid] if tid<len(pidv) else 9999. for pidv,tid in zip(trk_llr_pid_v,trk_id)])
    df['trkpid'] = trk_llr_pid_v_sel
    df['subcluster'] = df['shrsubclusters0'] + df['shrsubclusters1'] + df['shrsubclusters2']
    df['trkfit'] = df['shr_tkfit_npointsvalid'] / df['shr_tkfit_npoints']
    df['anglediff_Y'] = np.abs(df['secondshower_Y_dir']-df['shrclusdir2'])
    #df['FMdiffOBV'] = df['nu_flashmatch_score']-df['best_obviouscosmic_flashmatch_score']
    #df['FMdiff'] = df['nu_flashmatch_score']-df['best_cosmic_flashmatch_score']

In [ ]:
# define some energy-related variables
for i,df in enumerate(df_v):
    df["reco_e"] = (df["shr_energy_tot_cali"] + 0.030) / 0.79 + df["trk_energy_tot"]
    df["reco_e_qe"] = 0.938*((df["shr_energy"]+0.030)/0.79)/(0.938 - ((df["shr_energy"]+0.030)/0.79)*(1-np.cos(df["shr_theta"])))
    df["reco_e_rqe"] = df["reco_e_qe"]/df["reco_e"]

# and a way to filter out data
for i,df in enumerate(df_v):
    df["bnbdata"] = np.zeros_like(df["shr_energy"])
data["bnbdata"] = np.ones_like(data["shr_energy"])

# test calibrating dedx on Y plane
#data["shr_tkfit_gap10_dedx_Y"] = data["shr_tkfit_gap10_dedx_Y"]*1.03
#ext["shr_tkfit_gap10_dedx_Y"] = ext["shr_tkfit_gap10_dedx_Y"]*1.03
#data["shr_tkfit_2cm_dedx_Y"] = data["shr_tkfit_2cm_dedx_Y"]*1.03
#ext["shr_tkfit_2cm_dedx_Y"] = ext["shr_tkfit_2cm_dedx_Y"]*1.03
#data["shr_tkfit_dedx_Y"] = data["shr_tkfit_dedx_Y"]*1.03
#ext["shr_tkfit_dedx_Y"] = ext["shr_tkfit_dedx_Y"]*1.03


In [ ]:
# avoid double-counting of events out of FV in the NC/CC pi0 samples
ncpi0 = ncpi0.query('category != 5')
ccpi0 = ccpi0.query('category != 5')
ccnopi = ccnopi.query('category != 5')
nccpi = nccpi.query('category != 5')
ncnopi = ncnopi.query('category != 5')

# add back the cosmic category, for background only
df_v = [lee,mc,ncpi0,ccpi0,ccnopi,nccpi,ncnopi,nue,ext,data,dirt]
for i,df in enumerate(df_v):
    df.loc[(df['category']!=1)&(df['category']!=10)&(df['category']!=11)&(df['category']!=111)&(df['slnunhits']/df['slnhits']<0.5), 'category'] = 4

In [ ]:
lee["is_signal"] = lee["category"] == 11
data["is_signal"] = data["category"] == 11
nue["is_signal"] = nue["category"] == 11
mc["is_signal"] = mc["category"] == 11
dirt["is_signal"] = dirt["category"] == 11
ext["is_signal"] = ext["category"] == 11
ncpi0["is_signal"] = ncpi0["category"] == 11
ccpi0["is_signal"] = ccpi0["category"] == 11
ccnopi["is_signal"] = ccnopi["category"] == 11
nccpi["is_signal"] = nccpi["category"] == 11
ncnopi["is_signal"] = ncnopi["category"] == 11

lee.loc[lee['category'] == 1, 'category'] = 111
lee.loc[lee['category'] == 10, 'category'] = 111
lee.loc[lee['category'] == 11, 'category'] = 111

if (USEBDT == True):
    train_ncpi0, ncpi0 = train_test_split(ncpi0, test_size=0.5, random_state=1990)
    train_ccpi0, ccpi0 = train_test_split(ccpi0, test_size=0.5, random_state=1990)
    train_ccnopi, ccnopi = train_test_split(ccnopi, test_size=0.5, random_state=1990)
    train_nccpi, nccpi = train_test_split(nccpi, test_size=0.5, random_state=1990)
    train_ncnopi, ncnopi = train_test_split(ncnopi, test_size=0.5, random_state=1990)
    train_mc, mc = train_test_split(mc, test_size=0.5, random_state=1990)
    train_nue, nue = train_test_split(nue, test_size=0.5, random_state=1990)
    train_lee, lee = train_test_split(lee, test_size=0.5, random_state=1990)
    train_ext, ext = train_test_split(ext, test_size=0.5, random_state=1990)

In [ ]:
# variables to be trained on
TRAINVAR = [#"n_showers_contained",
            "shr_score","tksh_distance","tksh_angle",
            "shr_tkfit_gap10_dedx_Y","shr_tkfit_gap10_dedx_U","shr_tkfit_gap10_dedx_V",
            "shr_tkfit_2cm_dedx_Y","shr_tkfit_2cm_dedx_U","shr_tkfit_2cm_dedx_V",
            "trkfit","trkpid",
            "subcluster","shrmoliereavg",#"shrmoliererms",
            "trkshrhitdist2","hits_ratio",
            "secondshower_Y_nhit","secondshower_Y_vtxdist","secondshower_Y_dot","anglediff_Y",
            #'secondshower_V_nhit','secondshower_V_vtxdist','secondshower_V_dot','anglediff_V',
            #'secondshower_U_nhit','secondshower_U_vtxdist','secondshower_U_dot','anglediff_U',
            #"pi0_radlen1","pi0_radlen2","pi0_dedx2_fit_Y",
            #"pi0_energy2_Y","pi0_mass_Y","pi0_gammadot",
           ]

In [ ]:
LABELS = ['pi0','nonpi0']

if (USEBDT == True):
    #for label, bkg_query in zip(nue_booster.labels, nue_booster.bkg_queries):
    for label, bkg_query in zip(LABELS, nue_booster.bkg_queries):
        with open(ls.pickle_path+'booster_%s_dec.pickle' % label, 'rb') as booster_file:
            booster = pickle.load(booster_file)
            mc[label+"_score"] = booster.predict(
                xgb.DMatrix(mc[TRAINVAR]),
                ntree_limit=booster.best_iteration)
            nue[label+"_score"] = booster.predict(
                xgb.DMatrix(nue[TRAINVAR]),
                ntree_limit=booster.best_iteration)
            ext[label+"_score"] = booster.predict(
                xgb.DMatrix(ext[TRAINVAR]),
                ntree_limit=booster.best_iteration)
            data[label+"_score"] = booster.predict(
                xgb.DMatrix(data[TRAINVAR]),
                ntree_limit=booster.best_iteration)
            dirt[label+"_score"] = booster.predict(
                xgb.DMatrix(dirt[TRAINVAR]),
                ntree_limit=booster.best_iteration)
            lee[label+"_score"] = booster.predict(
                xgb.DMatrix(lee[TRAINVAR]),
                ntree_limit=booster.best_iteration)
            ncpi0[label+"_score"] = booster.predict(
                xgb.DMatrix(ncpi0[TRAINVAR]),
                ntree_limit=booster.best_iteration)
            ccpi0[label+"_score"] = booster.predict(
                xgb.DMatrix(ccpi0[TRAINVAR]),
                ntree_limit=booster.best_iteration)
            ccnopi[label+"_score"] = booster.predict(
                xgb.DMatrix(ccnopi[TRAINVAR]),
                ntree_limit=booster.best_iteration)
            nccpi[label+"_score"] = booster.predict(
                xgb.DMatrix(nccpi[TRAINVAR]),
                ntree_limit=booster.best_iteration)
            ncnopi[label+"_score"] = booster.predict(
                xgb.DMatrix(ncnopi[TRAINVAR]),
                ntree_limit=booster.best_iteration)


In [ ]:
dfbdt_v = [lee,mc,ncpi0,ccpi0,ccnopi,nccpi,ncnopi,nue,ext,data,dirt]

In [ ]:
import plotter
import importlib
importlib.reload(plotter)

In [ ]:
samples = {
    "mc": mc,
    "nue": nue,
    "data": data,
    "ext": ext,
    "dirt": dirt,
    "ncpi0": ncpi0,
    "ccpi0": ccpi0,
    "ccnopi": ccnopi,
    "nccpi": nccpi,
    "ncnopi": ncnopi,
    "lee": lee
}


if ISRUN3:
    scaling = 101.0/0.865
else:
    scaling = 101.0/4.07
#scaling = 1

SPLIT = 1.0
if (USEBDT == True):
    SPLIT = 2.

#''' 0109
if ISRUN3:
    weights = {
        "mc": 6.47e-3 * SPLIT * scaling,
        "nue": 1.19e-4 * SPLIT * scaling,
        "ext": 2.18e-2 * SPLIT * scaling,
        "dirt": 2.66e-2 * scaling,
        "lee": 1.19e-4 * SPLIT * scaling,
        "ncpi0": 3.79e-3 * SPLIT * scaling,
        "ccpi0": 1.34e-3 * SPLIT * scaling,
        #"ccnopi": 3.11e-03 * SPLIT * scaling,
        "ccnopi": 6.47e-3 * 0.45 * SPLIT * scaling, #temporary, until we fix pi0truth_elec_etot
        #"nccpi": 9.64e-04 * SPLIT * scaling,
        "nccpi": 6.47e-3 * 0.114 * SPLIT * scaling, #temporary, until we fix pion counting
        "ncnopi": 1.86e-03 * SPLIT * scaling,
    }
    pot = 0.865e19*scaling
else:
    weights = {
        "mc": 3.39e-2 * SPLIT * scaling,
        "nue": 7.95e-4 * SPLIT * scaling,
        "ext": 1.39e-1 * SPLIT * scaling,
        "dirt": 1.32e-1 * scaling,
        "lee": 7.95e-4 * SPLIT * scaling,
        "ncpi0": 1.56e-2 * SPLIT * scaling,
        "ccpi0": 1.21e-2 * SPLIT * scaling,
        #"ccnopi": 1.36e-02 * SPLIT * scaling,
        "ccnopi": 3.39e-2 * 0.45 * SPLIT * scaling, #temporary, until we fix pi0truth_elec_etot
        #"nccpi": 4.54e-03 * SPLIT * scaling,
        "nccpi": 3.39e-2 * 0.114 * SPLIT * scaling, #temporary, until we fix pion counting
        "ncnopi": 9.60e-03 * SPLIT * scaling,
    }
    pot = 4.07e19*scaling
#'''
my_plotter = plotter.Plotter(samples, weights, pot=pot)
print(weights)

In [ ]:
# giuseppe's box cuts
QUERY = "selected == 1"
QUERY += " and (crtveto!=1 or crthitpe < 100.) and (_closestNuCosmicDist > 20.)"
QUERY += " and (reco_nu_vtx_x>20 & reco_nu_vtx_x<230) "
QUERY += " and (reco_nu_vtx_y>-95 & reco_nu_vtx_y<95)"
QUERY += " and n_showers_contained==1"
QUERY += " and shr_energy_tot_cali>0.08"
QUERY += " and hits_y>50"
QUERY += " and hits_ratio>0.65"
QUERY += " and CosmicIP>30"
QUERY += " and shr_distance<2.5"
QUERY += " and trk_distance<2.5"
QUERY += " and tksh_distance<2"
QUERY += " and tksh_angle>-0.9"
QUERY += " and (shr_tkfit_dedx_Y>0.5 & shr_tkfit_dedx_Y<3.5)"
QUERY += " and shr_score<0.125"
QUERY += " and trk_score>0.85"
QUERY += " and slclustfrac>0.6"
#QUERY += " and reco_e_rqe>0.8"

In [ ]:
# muon selection
QUERY = 'nslice == 1'
#if ISRUN3: QUERY += ' and ((crtveto!=1) or (crthitpe < 100)) and (_closestNuCosmicDist > 20.)'
QUERY += ' and trk_len > 20'
QUERY += ' and topological_score > 0.06'
QUERY += ' and reco_nu_vtx_sce_x > 5 and reco_nu_vtx_sce_x < 251'
QUERY += ' and reco_nu_vtx_sce_y > -110 and reco_nu_vtx_sce_y < 110'
QUERY += ' and reco_nu_vtx_sce_z > 20 and reco_nu_vtx_sce_z < 986'
#QUERY += ' and bkpdg == 2212 and (trkcompl < 0.5 or trkpur < 0.5)'

print (QUERY)

In [ ]:
# nue pre-selection
QUERY = 'nslice == 1' # cut # 1
QUERY += ' and selected == 1'
QUERY += ' and n_tracks_contained > 0'
#if ISRUN3: QUERY += ' and (crtveto!=1) and (_closestNuCosmicDist > 20.)'
QUERY += ' and shr_energy_tot_cali > 0.07'

# in case you do not want to look at the data
#QUERY += ' and bnbdata==0'

# Loose box cuts
if True:
    QUERY += ' and n_showers_contained == 1'
    QUERY += ' and hits_ratio > 0.5'
    QUERY += ' and tksh_distance < 6.0'
    QUERY += ' and shr_tkfit_dedx_Y < 4.0'
    QUERY += ' and tksh_angle > -0.9'
    QUERY += ' and trkpid < 0.1'
    QUERY += ' and shr_score < 0.30'
    QUERY += ' and CosmicIP > 20.'

# Box cuts
if False:
    QUERY += ' and tksh_distance < 3.'
    QUERY += ' and (shr_tkfit_gap10_dedx_Y > 0 and shr_tkfit_gap10_dedx_Y < 4.0 and shr_tkfit_gap10_dedx_U < 4.0 and shr_tkfit_gap10_dedx_V < 4.0)'
    QUERY += ' and (shr_tkfit_2cm_dedx_Y > 0 and shr_tkfit_2cm_dedx_Y < 4.0 and shr_tkfit_2cm_dedx_U < 4.0 and shr_tkfit_2cm_dedx_V < 4.0)'
    QUERY += ' and tksh_angle > -0.9 and tksh_angle < 0.7'
    QUERY += ' and shrmoliereavg > 2 and shrmoliereavg < 9'
    QUERY += ' and trkpid < -0.05'
    QUERY += ' and n_showers_contained == 1'
    QUERY += ' and shr_score < 0.2'
    QUERY += ' and hits_ratio > 0.65'
    QUERY += ' and CosmicIP > 20.'
    QUERY += ' and (secondshower_Y_nhit<=8 or secondshower_Y_dot<=0.8 or anglediff_Y<=30 or secondshower_Y_vtxdist>=100)'
    QUERY += ' and (trkfit < 0.45 or subcluster > 6)'

##QUERY += ' and (shr_tkfit_dedx_Y > 0 and shr_tkfit_dedx_Y < 4.0 and shr_tkfit_dedx_U < 4.0 and shr_tkfit_dedx_V < 4.0)'
##QUERY += ' and ((shr_tkfit_dedx_Y > 1.5 and shr_tkfit_dedx_Y < 3.0) or (shr_tkfit_dedx_Y > 0 and shr_tkfit_dedx_Y < 3.5 and shr_tkfit_dedx_U < 3.5 and shr_tkfit_dedx_V < 3.5))'
##QUERY += ' and ((shr_tkfit_gap10_dedx_Y > 1.5 and shr_tkfit_gap10_dedx_Y < 3.0) or (shr_tkfit_gap10_dedx_Y > 0 and shr_tkfit_gap10_dedx_Y < 3.5 and shr_tkfit_gap10_dedx_U < 3.5 and shr_tkfit_gap10_dedx_V < 3.5))'
##QUERY += ' and ((shr_tkfit_2cm_dedx_Y > 1.5 and shr_tkfit_2cm_dedx_Y < 3.0) or (shr_tkfit_2cm_dedx_Y > 0 and shr_tkfit_2cm_dedx_Y < 3.5 and shr_tkfit_2cm_dedx_U < 3.5 and shr_tkfit_2cm_dedx_V < 3.5))'

# Box cuts

#QUERY += ' and tksh_distance < 2.0'#3.0
#QUERY += ' and ((shr_tkfit_dedx_Y > 1.5 and shr_tkfit_dedx_Y < 3.0) or (shr_tkfit_dedx_Y > 0 and shr_tkfit_dedx_Y < 3.0 and shr_tkfit_dedx_U < 3.0 and shr_tkfit_dedx_V < 3.0))'
#QUERY += ' and tksh_angle > -0.9 and tksh_angle < 0.5'
#QUERY += ' and trkfit < 0.45'
#QUERY += ' and shrmoliereavg < 10'
#QUERY += ' and trkpid < 0.0'
#QUERY += ' and n_showers_contained == 1'
#QUERY += ' and shr_score < 0.10'#0.15
#QUERY += ' and hits_ratio>0.7'
#QUERY += ' and CosmicIP > 20.'
#QUERY += ' and (secondshower_Y_nhit<=7 or secondshower_Y_dot<=0.8 or (anglediff_Y<=20 and anglediff_Y>=-20.) or secondshower_Y_vtxdist>=100)'

#QUERY += ' and ~(secondshower_Y_nhit>7 and secondshower_Y_dot>0.8 and (anglediff_Y>20 or anglediff_Y<-20.) and secondshower_Y_vtxdist<100)'
#QUERY += ' and (secondshower_V_nhit<=7 or secondshower_V_dot<=0.8 or (anglediff_V<=20 and anglediff_V>=-20.) or secondshower_V_vtxdist>=100)'
#QUERY += ' and (secondshower_U_nhit<=7 or secondshower_U_dot<=0.8 or (anglediff_U<=20 and anglediff_U>=-20.) or secondshower_U_vtxdist>=100)'

# BDT cuts

if True:
    QUERY += ' and pi0_score > 0.995'
    QUERY += ' and nonpi0_score > 0.9984'

#QUERY += ' and cc_score > 0.99'
#QUERY += ' and cosmic_score > 0.96'
#QUERY += ' and ext_score > 0.985'
#QUERY += ' and ccpi0_score > 0.97'
#QUERY += ' and ncpi0_score > 0.96'

In [ ]:
# 1eNp box-cuts
# nue pre-selection
QUERY = 'nslice == 1'
QUERY += ' and selected == 1'
QUERY += ' and n_tracks_contained > 0'
QUERY += ' and shr_energy_tot_cali > 0.07'
QUERY += ' and n_tracks_contained > 0'
# Loose box cuts
QUERY += ' and n_showers_contained == 1'
QUERY += ' and hits_ratio > 0.5'
QUERY += ' and tksh_distance < 6.0'
QUERY += ' and shr_tkfit_2cm_dedx_Y < 4.0'
QUERY += ' and tksh_angle > -0.9'
QUERY += ' and trkpid < 0.1'
QUERY += ' and shr_score < 0.30'
QUERY += ' and CosmicIP > 20.'
# Box cuts
QUERY += ' and tksh_distance < 3.5'
QUERY += ' and (shr_tkfit_gap10_dedx_Y > 0 and shr_tkfit_gap10_dedx_Y < 4.5 and shr_tkfit_gap10_dedx_U < 4.5 and shr_tkfit_gap10_dedx_V < 4.5)'
QUERY += ' and (shr_tkfit_2cm_dedx_Y > 0 and shr_tkfit_2cm_dedx_Y < 4.0 and shr_tkfit_2cm_dedx_U < 4.0 and shr_tkfit_2cm_dedx_V < 4.0)'
QUERY += ' and tksh_angle > -0.9 and tksh_angle < 0.7'
QUERY += ' and shrmoliereavg > 2 and shrmoliereavg < 9'
QUERY += ' and trkpid < -0.02'
QUERY += ' and n_showers_contained == 1'
QUERY += ' and shr_score < 0.275'
QUERY += ' and hits_ratio > 0.60'
QUERY += ' and CosmicIP > 20.'
QUERY += ' and (secondshower_Y_nhit<=8 or secondshower_Y_dot<=0.8 or anglediff_Y<=40 or secondshower_Y_vtxdist>=100)'
QUERY += ' and (trkfit < 0.45 or subcluster > 6)'

# in case you do not want to look at the data
QUERY += ' and bnbdata==0'

In [ ]:
# nue pre-selection
QUERY = 'nslice == 1'
QUERY += ' and selected == 1'
QUERY += ' and n_tracks_contained > 0'
QUERY += ' and shr_energy_tot_cali > 0.07'
#if ISRUN3: QUERY += ' and (crtveto!=1) and (_closestNuCosmicDist > 20.)'
#QUERY += ' and bnbdata==0'

In [ ]:
# 1eNp BDT
QUERY = 'nslice == 1'
QUERY += ' and selected == 1'
QUERY += ' and n_tracks_contained > 0'
QUERY += ' and shr_energy_tot_cali > 0.07'
#if ISRUN3: QUERY += ' and (crtveto!=1) and (_closestNuCosmicDist > 20.)'
    
# Loose box cuts
QUERY += ' and n_showers_contained == 1'
QUERY += ' and hits_ratio > 0.5'
QUERY += ' and tksh_distance < 6.0'
QUERY += ' and shr_tkfit_2cm_dedx_Y < 4.0'
QUERY += ' and tksh_angle > -0.9'
QUERY += ' and trkpid < 0.1'
QUERY += ' and shr_score < 0.30'
QUERY += ' and CosmicIP > 20.'  

# in case you do not want to look at the data
QUERY += ' and bnbdata==0'
    
# and BDT    
#QUERY += ' and pi0_score > 0.995 and nonpi0_score > 0.9984'
#QUERY += ' and pi0_score > 0.9 and nonpi0_score > 0.9'

In [ ]:
# create output files based on selection
fname_v = [NUE,NU,PI0,NUE,EXT,BNB,DRT]

for i,name in enumerate(fname_v):
    if (i == 0):
        continue # skip LEE event. This one will be done within SBNFit
        
    df = dfbdt_v[i]

    fout = open(ls.ntuple_path+name+'.txt','w')
    dfsel = df.query(QUERY)
    print ('file %s has %i selected entries'%(name,dfsel.shape[0]))
    for i,row in dfsel.iterrows():
        run = row['run'].values[0]
        sub = row['sub'].values[0]
        evt = row['evt'].values[0]
        fout.write('%i %i %i \n'%(run,sub,evt))
    fout.close()

In [ ]:
miniboone_bins = [
    0, 0.200, 0.300, 0.375, 0.475, 0.550, 0.675, 0.800, 0.950, 1.100, 1.300,
    1.500, 3.000
]

#QUERY += " & pi0truth_elec_etot<15 & npi0 == 0 & ccnc == 0"
#QUERY += " & muon_e>0.3 & npi0 == 0 & ccnc == 0"
#QUERY = "(muon_e<0.3 | pi0truth_elec_etot>=15) & npi0 == 0 & npion == 0 & ccnc == 0 & nu_pdg == 14 & category == 2"
#QUERY += " & ~((muon_e<0.3 | pi0truth_elec_etot>=15) & npi0 == 0 & npion == 0 & ccnc == 0 & nu_pdg == 14 & category != 5)"

#VARIABLE, BINS, RANGE, XTIT = 'reco_e',1,(0.2,1.),r"Reconstructed Energy [GeV]"
VARIABLE, BINS, RANGE, XTIT = 'reco_e',14,(0.1,1.5),r"Reconstructed Energy [GeV]"
#VARIABLE, BINS, RANGE, XTIT = 'pi0_mass_Y',20,(0,500),"pi0 mass Y"

#VARIABLE, BINS, RANGE, XTIT = 'muon_e',20,(0.,1.),r"Muon Energy [GeV]"
#VARIABLE, BINS, RANGE, XTIT = 'pi0truth_elec_etot',20,(0.,60),r"Michel Energy [GeV]"

fig, ax1, ax2 = my_plotter.plot_variable(
    VARIABLE,   
    query=QUERY,
    kind="event_category",
    #kind="interaction",
    #kind="sample",
    #kind='particle_pdg',
    draw_sys=False,
    title=XTIT,
    #bins=asymm_bins,
    bins=BINS,
    range=RANGE,
)

print("Profile likelihood: %.2f sigma @ 1.01e21 POT" % my_plotter.significance_likelihood)
print("s/sqrt(b): %.2f sigma @ 1.01e21 POT" % my_plotter.significance)

#ax1.set_ylim(0,40)
#ax1.set_yscale("log")
#ax1.set_ylim(0,200)
ax1.set_ylim(0,ax1.get_ylim()[1]*1.4)
#ax2.set_ylim(0.5,1.5)
#fig.savefig(ls.plots_path+"/0109/run3/presel/"+VARIABLE+"_"+date_time+".pdf")

In [ ]:
data.query(QUERY)[["run","sub","evt","reco_e"]]

In [ ]:
QUERY += ' and reco_e > 0.2 and reco_e < 1.5'
#QUERY += ' and shr_tkfit_dedx_Y < 0'
plots = []
plots.append(['tksh_distance',20,(0,10),"tksh distance [cm]"])
#plots.append(['shr_tkfit_dedx_Y',20,(0,10),"shr tkfit dE/dx (Y, 0-4 cm) [MeV/cm]"])
#plots.append(['shr_tkfit_dedx_U',20,(0,10),"shr tkfit dE/dx (U, 0-4 cm) [MeV/cm]"])
#plots.append(['shr_tkfit_dedx_V',20,(0,10),"shr tkfit dE/dx (V, 0-4 cm) [MeV/cm]"])
plots.append(['shr_tkfit_2cm_dedx_Y',20,(0,10),"shr tkfit dE/dx (Y, 0-2 cm) [MeV/cm]"])
plots.append(['shr_tkfit_2cm_dedx_U',20,(0,10),"shr tkfit dE/dx (U, 0-2 cm) [MeV/cm]"])
plots.append(['shr_tkfit_2cm_dedx_V',20,(0,10),"shr tkfit dE/dx (V, 0-2 cm) [MeV/cm]"])
plots.append(['shr_tkfit_gap10_dedx_Y',20,(0,10),"shr tkfit dE/dx (Y, 1-5 cm) [MeV/cm]"])
plots.append(['shr_tkfit_gap10_dedx_U',20,(0,10),"shr tkfit dE/dx (U, 1-5 cm) [MeV/cm]"])
plots.append(['shr_tkfit_gap10_dedx_V',20,(0,10),"shr tkfit dE/dx (V, 1-5 cm) [MeV/cm]"])
plots.append(['tksh_angle',20,(-1,1),"cos(tksh angle)"])
plots.append(['trkfit',20,(0,1),"Fraction of Track-fitted points"])
plots.append(['shrmoliereavg',20,(0,50),"average Moliere angle [degrees]"])
plots.append(['trkpid',15,(-1,1),"track LLR PID"])
plots.append(['n_showers_contained',10,(0,10),"n showers contained"])
plots.append(['n_tracks_contained',10,(0,10),"n tracks contained"])
plots.append(['shr_score',20,(0,0.5),"shr score"])
plots.append(['trk_score',20,(0.5,1.0),"trk score"])
plots.append(['hits_ratio',20,(0,1),"shower hits/all hits"])
plots.append(['slclustfrac',20,(0,1),"slice clustered fraction"])
plots.append(['CosmicIP',20,(0,200),"Cosmic IP from vtx [cm]"])
plots.append(['reco_nu_vtx_x',20,(0,260),"x"])
plots.append(['reco_nu_vtx_y',20,(-120,120),"y"])
plots.append(['reco_nu_vtx_z',20,(0,1100),"z"])
plots.append(['reco_e_rqe',20,(0,2),"rqe"])
plots.append(['trkshrhitdist2',20,(0,10),"2D trk-shr distance (Y)"])
plots.append(['subcluster',20,(0,20),"N sub-clusters in shower"])
plots.append(['shrmoliererms',20,(0,5000),"RMS Moliere angle [degrees]"])
plots.append(['shr_energy_tot_cali',20,(0,1),"shr energy (calibrated) [GeV]"])
plots.append(['contained_fraction',20,(0.,1),"contained fraction"])
plots.append(['hits_y',20,(0.,1000),"N hits Y plane"])
#plots.append(['pi0_mass_Y',25,(-50.,200),"pi0 mass [Mev]"])
#plots.append(['pi0_gammadot',20,(-1,1),"2-shower angle"])
plots.append(['secondshower_Y_nhit',20,(0,200),"Nhit 2nd shower (Y)"])
plots.append(['secondshower_Y_dot',20,(-1,1),"cos(2nd shower direction wrt vtx) (Y)"])
plots.append(['anglediff_Y',20,(0,350),"angle diff 1st-2nd shower (Y) [degrees]"])
plots.append(['secondshower_Y_vtxdist',20,(0.,200),"vtx dist 2nd shower (Y)"])
if USEBDT:
    plots = []
    plots.append(['pi0_score',20,(0.,1),"$\pi^0$ BDT response"])
    plots.append(['nonpi0_score',20,(0.,1),"non-$\pi^0$ BDT response"])
    #plots.append(['cc_score',40,(0.,1),"cc score"])
    #plots.append(['cosmic_score',40,(0.,1),"cosmic score"])
    #plots.append(['ext_score',40,(0.,1),"ext score"])
    #plots.append(['ccpi0_score',40,(0.,1),"ccpi0 score"])
    #plots.append(['ncpi0_score',40,(0.,1),"ncpi0 score"])
for VARIABLE, BINS, RANGE, XTIT in plots:
    print(VARIABLE, BINS, RANGE, XTIT)
    fig, ax1, ax2 = my_plotter.plot_variable(
        VARIABLE,   
        query=QUERY,
        kind="event_category",
        #kind="interaction",
        #kind="sample",
        draw_sys=False,
        title=XTIT,
        bins=BINS,
        range=RANGE,
    )
    #ax1.set_ylim(0,ax1.get_ylim()[1]*1.4)
    ax1.set_yscale("log")
    fig.show()
    #fig.savefig(ls.plots_path+"0109/run1/presel/"+VARIABLE+date_time+".pdf")

In [ ]:
import matplotlib.cm as cm
from matplotlib.colors import Normalize
import pandas as pd
import matplotlib.pyplot as plt

BINS = np.linspace(0,1,25)
VAR = 'cosmic_score'

nueplot = nue.query(QUERY)
nueplotecut = nueplot.query('nu_e < 0.8')

fig = plt.figure(figsize=(6,6))
plt.hist(nueplot[VAR].values,bins=BINS,histtype='step',lw=2,color='red',label='all nue')
plt.hist(nueplotecut[VAR].values,bins=BINS,histtype='step',lw=2,color='m',label='nue < 0.8 GeV')
#plt.hist(nue[VAR].values,bins=BINS,histtype='step',lw=2,color='green')
plt.grid()
plt.legend()
plt.title(VAR)
plt.show()

BINS2D = (np.linspace(0,3.0,25),BINS)

fig = plt.figure(figsize=(6,6))
plt.hist2d(nueplot['nu_e'].values,nueplot[VAR].values,bins=BINS2D)
#plt.hist(nue[VAR].values,bins=BINS,histtype='step',lw=2,color='green')
plt.grid()
plt.show()

In [ ]:
import matplotlib.cm as cm
from matplotlib.colors import Normalize
import pandas as pd
import matplotlib.pyplot as plt

x_range = (0.1,0.8)
n_bins = 7

n_tot = np.empty([50, n_bins])
n_cv_tot = np.empty(n_bins)
n_tot.fill(0)
n_cv_tot.fill(0)
score_cut = 0.997

my_cmap = cm.get_cmap('viridis')

for t in samples:
    if t in ["ext", "data", "lee"]:
        continue
    tree = samples[t]
    
    extra_query = ""
    if t == "mc":
        extra_query = "& ~(nu_pdg == 12 & ccnc == 0) & ~(npi0 != 0 & ccnc == 1)"
        
    queried_tree = tree.query("selected==1 & global_score > %g %s & interaction == 1" % (score_cut, extra_query))
    variable = queried_tree["reco_e"]
    genie_weights = queried_tree["weightsGenie"]
    spline_fix = queried_tree["weightSpline"]*weights[t]

    s = genie_weights
    df = pd.DataFrame(s.values.tolist())


    n_cv, bins = np.histogram(
        variable,
        range=x_range,
        bins=n_bins,
        weights=spline_fix)
    
    n_cv_tot += n_cv

    if not df.empty:
        for i in range(50):
            weight = df[i].values
            weight[np.isnan(weight)] = 1
            weight[weight > 100] = 1

            n, bins = np.histogram(
                variable, weights=weight*spline_fix, range=x_range, bins=n_bins)

            n_tot[i] += n

        
bincenters = 0.5 * (bins[1:] + bins[:-1])

left,right = bins[:-1],bins[1:]
X = np.array([left,right]).T.flatten()
Y = np.array([n_cv_tot,n_cv_tot]).T.flatten()
plt.plot(X,Y,color='r',linewidth=2,label='Central value')

    
cov = np.empty([len(n_cv), len(n_cv)])
cov.fill(0)

my_norm = Normalize(vmin=min((sum(n) for n in n_tot)), vmax=max((sum(n) for n in n_tot)))

for n in n_tot:
    left,right = bins[:-1],bins[1:]
    X = np.array([left,right]).T.flatten()
    Y = np.array([n,n]).T.flatten()
    plt.plot(X,Y,color=my_cmap(my_norm(sum(n))),zorder=-32)
    for i in range(len(n_cv)):
        for j in range(len(n_cv)):
            cov[i][j] += (n[i] - n_cv_tot[i]) * (n[j] - n_cv_tot[j])

cov /= 50
            
frac_cov = np.empty([len(n_cv), len(n_cv)])
corr = np.empty([len(n_cv), len(n_cv)])

for i in range(len(n_cv)):
    for j in range(len(n_cv)):
        frac_cov[i][j] = cov[i][j] / (n_cv_tot[i] * n_cv_tot[j])
        corr[i][j] = cov[i][j] / np.sqrt(cov[i][i] * cov[j][j])

        
plt.errorbar(
    bincenters, 
    n_cv_tot,
    yerr=np.sqrt(np.diag(cov)), 
    fmt='none', 
    ecolor='r',
    linewidth=2,
    label='GENIE sys. uncertainties')

print(np.sqrt(np.diag(cov))/n_cv_tot)
plt.xlim(x_range[0], x_range[1])
plt.xlabel(r"$E_{deposited}$ [GeV]")
plt.legend()
plt.ylim(ymin=0)
plt.tight_layout()
plt.savefig("plots/spec.pdf", transparent=True)

In [ ]:
fig, ax = plt.subplots(1, 1)
pos = ax.imshow(frac_cov, origin='lower', cmap='viridis')
ax.set_title("Fractional covariance matrix")
ax.set_ylabel("Bin number")
ax.set_xlabel("Bin number")
fig.colorbar(pos)
fig.tight_layout()
fig.savefig("plots/frac.pdf", transparent=True)

In [ ]:
fig, ax = plt.subplots(1, 1)
ax.set_title("Correlation matrix")
pos = ax.imshow(corr, origin='lower', cmap='inferno')
ax.set_ylabel("Bin number")
ax.set_xlabel("Bin number")
fig.colorbar(pos)
fig.tight_layout()
fig.savefig("plots/corr.pdf", transparent=True)

In [ ]:
my_plotter.plot_2d(
    "global_score",
    "ncpi0_score",
    query="selected==1",
    bins_x=50,
    bins_y=50,
    range_x=(0., 1),
    range_y=(0., 1))